## Test accuracy on 20 CIFAR-100 classes using MobileNet

### Import all the necessary libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

###  Load CIFAR-100 fine labels (100-class)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [6]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [7]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 128x128

In [8]:
x_train_20 = tf.image.resize(x_train_20, (128, 128)).numpy()
x_test_20 = tf.image.resize(x_test_20, (128, 128)).numpy()

2025-05-25 01:53:43.063774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-25 01:53:43.064376: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-05-25 01:53:43.121505: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1966080000 exceeds 10% of free system memory.


### Preprocess

In [9]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [10]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load MobileNet base model

In [ ]:
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # freeze base for transfer learning

# 8. Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(20, activation='softmax')
])

# 9. Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Train
model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

# 11. Evaluate
loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using NSANetMobile:", acc)

19993432/19993432 [==============================] - 6s 0us/step
Epoch 1/5


2025-05-25 01:54:12.287122: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1769472000 exceeds 10% of free system memory.


141/141 [==============================] - 59s 361ms/step - loss: 1.4271 - accuracy: 0.5900 - val_loss: 0.9388 - val_accuracy: 0.7120
Epoch 2/5
141/141 [==============================] - 47s 335ms/step - loss: 0.8467 - accuracy: 0.7437 - val_loss: 0.8427 - val_accuracy: 0.7350
Epoch 3/5
141/141 [==============================] - 47s 332ms/step - loss: 0.6937 - accuracy: 0.7828 - val_loss: 0.8105 - val_accuracy: 0.7460
Epoch 4/5
141/141 [==============================] - 47s 333ms/step - loss: 0.5984 - accuracy: 0.8107 - val_loss: 0.7962 - val_accuracy: 0.7460
Epoch 5/5
63/63 [==============================] - 10s 156ms/step - loss: 0.6602 - accuracy: 0.7965
Test accuracy on 20 CIFAR-100 classes using MobileNet: 0.796500027179718
